# Exploring Legacy Chains in LangChain

## Install OpenAI, and LangChain dependencies

In [ ]:
!pip install langchain==0.2.0
!pip install langchain-openai==0.1.7
!pip install langchain-community==0.2.0

## Enter Open AI API Key

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Setup Environment Variables

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

## Load Connection to LLM

Here we create a connection to ChatGPT to use later in our chains

In [ ]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

## Working with LangChain Chains

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components. Also running on multiple data points can be done easily with chains.

Chain's are the legacy interface for "chained" applications. We define a Chain very generically as a sequence of calls to components, which can include other chains.

Here we use LangChain's Legacy Chains which are going to get ported over to LCEL chains over time

### Working with LLMChain

The most common type of chaining in any LLM application is combining a prompt template with an LLM and optionally an output parser.

An `LLMChain` is a simple chain that adds some functionality around language models. It is used widely throughout LangChain, including in other chains and agents.

An `LLMChain` consists of a PromptTemplate and a language model (either an LLM or chat model). It formats the prompt template using the input key values provided (and also memory key values, if available), passes the formatted string to LLM and returns the LLM output.

__Note:__ `LLMChain` has been deprecated by LangChain in favor of using an LCEL variant, we will be looking at that in a subsequent demo on LCEL chains

In [ ]:
reviews = [
    f"""
    Purchased this adorable koala plush toy for my nephew's birthday,
    and he's absolutely smitten with it, carrying it around everywhere he goes.
    The plush is incredibly soft, and the koala's face has an endearing expression.
    However, I did find it a tad on the smaller side given its price point.
    I believe there may be larger alternatives available at a similar price.
    To my delight, it arrived a day earlier than anticipated,
    allowing me to enjoy it briefly before gifting it to him.
    """,
    f"""
    Required a stylish lamp for my office space, and this particular one
    came with added storage at a reasonable price.
    The delivery was surprisingly quick, arriving within just two days.
    However, the pull string for the lamp suffered damage during transit.
    To my relief, the company promptly dispatched a replacement,
    which arrived within a few days. Assembly was a breeze.
    Then, I encountered an issue with a missing component,
    but their support team responded swiftly and provided the missing part.
    It appears to be a commendable company that genuinely values its
    customers and the quality of its products.
    """
    ]

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import ChatPromptTemplate

prompt = """
            Act as a product review analyst.
            Your task is to generate a short summary of a product
            review from an ecommerce site.

            Generate a summary of the review (max 2 lines)
            Also show both the positives and negatives from the review (max 2 bullets)

            ```{review}```
"""

prompt_template = ChatPromptTemplate.from_template(prompt)
llm_chain = LLMChain(llm=chatgpt, prompt=prompt_template)

In [ ]:
reviews[0]

In [ ]:
result = llm_chain.invoke({'review': reviews[0]})

In [ ]:
result

In [ ]:
print(result['text'])

In [ ]:
formatted_reviews = [{'review': review}
                        for review in reviews]

results = llm_chain.map().invoke(formatted_reviews)
len(results)

In [ ]:
for result in results:
    print(result['text'])
    print()

### Combining Multiple Tasks with SequentialChain

The next step after calling a language model is to make a series of calls to a language model. This is particularly useful when you want to take the output from one call and use it as the input to another.

Sequential chains allow you to connect multiple chains and compose them into pipelines that execute some specific scenario.

Here given a few IT Support issues, for each customer issue:
- We want to detect the customer message language
- We want to translate the customer message from the source language to English
- We want the AI to generate a suitable response to the problem in English
- We want to translate this response to the customer language

In [ ]:
it_support_queue = [
    "I can't access my email. It keeps showing an error message. Please help.",
    "Tengo problemas con la VPN. No puedo conectarme a la red de la empresa. ¿Pueden ayudarme, por favor?",
    "Mon imprimante ne répond pas et n'imprime plus. J'ai besoin d'aide pour la réparer.",
    "我无法访问公司的网站。每次都显示错误信息。请帮忙解决。"
]

it_support_queue

Here we can use `SequentialChain` and chain multiple `LLMChains` where each chain solves one of the tasks using a specific prompt as follows:

In [ ]:
# Chain 1: Detect customer message language
prompt1 = """
  Act as a customer support agent.
  For the customer support message delimited below by triple backticks,
  Output the language of the message in one word only, e.g. Spanish

  Customer Message:
  ```{orig_msg}```
"""
prompt_template1 = ChatPromptTemplate.from_template(prompt1)
chain1 = LLMChain(llm=chatgpt, prompt=prompt_template1, output_key="orig_lang")

# Chain 2: Translate Customer Message to English
prompt2 = """
  Act as a customer support agent.
  For the customer message and customer message language delimited below by triple backticks,
  Translate the customer message from the customer message language to English
  if customer message language is not in English,
  else return back the original customer message.

  Customer Message:
  ```{orig_msg}```
  Customer Message Language:
  ```{orig_lang}```
"""
prompt_template2 = ChatPromptTemplate.from_template(prompt2)
chain2 = LLMChain(llm=chatgpt, prompt=prompt_template2, output_key="trans_msg")

# Chain 3: Generate a resolution response in English
prompt3 = """
  Act as a customer support agent.
  For the customer support message delimited below by triple backticks,
  Generate an appropriate resolution response in English.

  Customer Message:
  ```{trans_msg}```
"""
prompt_template3 = ChatPromptTemplate.from_template(prompt3)
chain3 = LLMChain(llm=chatgpt, prompt=prompt_template3, output_key="trans_response")

# Chain 4: Translate resolution response from English to Customer's language
prompt4 = """
  Act as a customer support agent.
  For the customer resolution response and target language delimited below by triple backticks,
  Translate the customer resolution response message from English to the target language
  if target language is not in English,
  else return back the original customer resolution response.

  Customer Resolution Response:
  ```{trans_response}```
  Target Language:
  ```{orig_lang}```
"""
prompt_template4 = ChatPromptTemplate.from_template(prompt4)
chain4 = LLMChain(llm=chatgpt, prompt=prompt_template4, output_key="response")

In [ ]:
from langchain.chains import SequentialChain

# combine all chains sequentially
seq_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["orig_msg"], # input data
    output_variables=["orig_msg", "orig_lang","trans_msg", "response", "trans_response"], # response data
    verbose=True # set to False to turn of debugging messages
)

In [ ]:
support_msgs_formatted = [{'orig_msg': msg} for msg in it_support_queue]
support_msgs_formatted

In [ ]:
# run the chain on all the support messages
results = seq_chain.map().invoke(support_msgs_formatted)

In [ ]:
results[2]

In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df

Other popular Legacy Chains include:

- `ConversationChain`
- `ConversationalRetrievalChain`
- `RetrievalQA`

However given LangChain's focus on LCEL Chains, we will be focusing on implementing all of these using LCEL Chains soon.